## 使用networkx计算PageRank

In [1]:
import networkx as nx

# 创建一个有向图
G = nx.DiGraph()

### 读取mysql数据库

In [2]:
import pymysql
cnx = pymysql.connect(host='localhost', user='root', password='123qwe12')
cursor = cnx.cursor()
cnx.select_db('IR_db')

In [3]:
def get_douban_url_as_node():
    # 创建SQL查询
    sql = "SELECT url FROM douban"
    # 执行查询
    cursor.execute(sql)
    # 获取所有的结果
    results = cursor.fetchall()
    # 将结果从元组列表转换为普通列表，并添加到图中作为节点
    for result in results:
        G.add_node(result[0])


In [4]:

def get_ids_from_same_title():
    # 创建SQL查询
    sql = "SELECT page_id FROM same_title"
    # 执行查询
    cursor.execute(sql)
    # 获取所有的结果
    results = cursor.fetchall()
    # 将结果从元组列表转换为普通列表
    ids_same_title = [result[0] for result in results]
    return ids_same_title

def get_page_url_as_node():
    global ids_page
    # 创建SQL查询
    sql = "SELECT id,url FROM page"
    # 执行查询
    cursor.execute(sql)
    # 获取所有的结果
    results = cursor.fetchall()
    # 将结果从元组列表转换为普通列表
    ids = [result[0] for result in results]
    urls = [result[1] for result in results]
    # 获取相同title的id
    ids_same_title = get_ids_from_same_title()
    print(len(ids_same_title))
    # 去除相同title的id
    for id,url in zip(ids,urls):
        if id not in ids_same_title:
            G.add_node(url)

    # 从same_title中按title分组，获取每组的第一个id
    sql = "SELECT MIN(page_id) FROM same_title GROUP BY title"
    cursor.execute(sql)
    results = cursor.fetchall()
    ids = [result[0] for result in results]
    print(len(ids))
    # 找到这些id对应的url
    for id in ids:
        sql = "SELECT url FROM page WHERE id = %d" % id
        cursor.execute(sql)
        results = cursor.fetchall()
        url = results[0]
        G.add_node(url)

### 创建节点 × 没必要了，创建边的时候会自动创建节点

In [5]:
get_douban_url_as_node()

In [6]:
get_page_url_as_node()

1435
610


### 创建节点之间的边

In [7]:
def get_douban_edges():
    # 从数据库中获取所有的记录
    sql = "SELECT url, links FROM douban"
    cursor.execute(sql)
    results = cursor.fetchall()
     # 对于每一条记录
    for url, links in results:
        # 将links按照'\n'解析成一个列表
        links = links.split('\n')
        # 对于每一个link
        for link in links:
           G.add_edge(url, link)

In [9]:
get_douban_edges()

In [8]:
def create_url_to_url_mapping():
    # 创建一个空的映射
    url_to_url = {}
    # 从same_title表中按title分组，获取每组的第一个URL
    sql = "SELECT MIN(page_id), title FROM same_title GROUP BY title"
    cursor.execute(sql)
    results = cursor.fetchall()
    # 将每个title的所有URL映射到对应的节点
    for page_id, title in results:
        # 获取该title的第一个URL
        sql = "SELECT url FROM same_title WHERE page_id = %d" % page_id
        cursor.execute(sql)
        results = cursor.fetchall()
        target_url = results[0]
        # 获取该title的所有URL
        sql = "SELECT url FROM same_title WHERE title = %s"
        cursor.execute(sql,(title,))
        urls = cursor.fetchall()
        for url in urls:
            url_to_url[url] = target_url
    return url_to_url

In [10]:
# 拿到同title的url映射
url_to_url = create_url_to_url_mapping()

In [11]:
def get_page_edges():
    # 从数据库中获取所有的记录
    sql = "SELECT url, links FROM page"
    cursor.execute(sql)
    results = cursor.fetchall()
    # 对于每一条记录
    for url, links in results:
        # 如果URL在映射中，使用映射中的值替换URL
        url = url_to_url.get(url, url)
        # 将links按照'\n'解析成一个列表
        links = links.split('\n')
        # 对于每一个link
        for link in links:
            # 如果link在映射中，使用映射中的值替换link
            link = url_to_url.get(link, link)
            # 在图G中添加一条从url到link的边
            G.add_edge(url, link)

In [12]:
get_page_edges()

### 计算PageRank

In [13]:
def cal_page_rank():
    # 使用networkx中的pagerank函数计算PR值
    pr = nx.pagerank(G)
    return pr

### 更新es索引

In [14]:
from elasticsearch import Elasticsearch

# 实例化es
es = Elasticsearch(hosts="http://localhost:9200")

In [39]:
# 使用URL搜索文档
res = es.search(index="web", body={"query": {"match": {"url": 'https://www.runoob.com/'}}})
# 查看page_id
page_id = res['hits']['hits'][0]['_source']['pageRank']
print(page_id)

0.0021668640962373142


In [15]:
from elasticsearch.helpers import scan

def add_page_rank_to_documents(pr):
    # 遍历web索引中的所有文档
    for doc in scan(es, index='web'):
        # 获取文档的URL
        url = doc['_source']['url']
        # 在pr字典中查询对应的PageRank值
        page_rank = pr.get(url)
        # 如果找到了PageRank值
        if page_rank is not None:
            # 更新文档，添加pageRank字段
            es.update(index='web', id=doc['_id'], body={'doc': {'pageRank': page_rank}})
        else :
            # 否则，设置pageRank字段为0
            es.update(index='web', id=doc['_id'], body={'doc': {'pageRank': 0}})

In [16]:
pr = cal_page_rank()

In [17]:
add_page_rank_to_documents(pr)

In [18]:
## 打印PageRank值
# 获取web索引中的前一百条文档
res = es.search(index="web", body={"query": {"match_all": {}}}, size=100)

# 遍历这些文档
for doc in res['hits']['hits']:
    # 获取并打印文档的title和pageRank字段
    title = doc['_source'].get('title', 'Not available')
    page_rank = doc['_source'].get('pageRank', 'Not available')
    print(f"Title: {title}, PageRank: {page_rank}")

Title: 从心理学的角度来谈谈, PageRank: 3.8256922927265375e-05
Title: 谁才是那个真正的看不见的客人？, PageRank: 3.8256922927265375e-05
Title: 《看不见的客人》中有哪些高明的细节？, PageRank: 3.8256922927265375e-05
Title: 细节是一切的关键――关于本片剧情逻辑的思考, PageRank: 3.8256922927265375e-05
Title: 这是今年你能在影院里看到的最穷的电影。, PageRank: 3.8256922927265375e-05
Title: 看不见的客人 Contratiempo, PageRank: 0.0005824634839805538
Title: 孙子，别装了, PageRank: 2.3971626135516338e-05
Title: 《无耻混蛋》:关于电影的电影, PageRank: 2.3971626135516338e-05
Title: 疯癫的诗意, PageRank: 2.3971626135516338e-05
Title: 谁才是混蛋？我还真不知道……, PageRank: 2.3971626135516338e-05
Title: 小游戏, PageRank: 2.3971626135516338e-05
Title: 杰作·未完成, PageRank: 2.3971626135516338e-05
Title: 我就乐意这样低俗了, PageRank: 2.3971626135516338e-05
Title: 犹太版抗日奇侠, PageRank: 2.3971626135516338e-05
Title: 《无耻混蛋》：代表人民爆你的头, PageRank: 2.3971626135516338e-05
Title: 无耻混蛋 Inglourious Basterds, PageRank: 0.00023483768004152637
Title: 多数现代人对《哈姆雷特》的理解，也就是《狮子王》了吧, PageRank: 2.3473086367071186e-05
Title: 没有the lion king，我的生活会是另一样, PageRank: 2.347308636

### 测试pageRank影响排序结果

In [18]:
def deal_bool_query(query):
    # 执行搜索
    response = es.search(
        index="web",
        body={
            "query": {
                "function_score": {
                    "query": {
                        "bool": {
                            "must": [
                                {
                                    "multi_match": {
                                        "query": query,
                                        "fields": ["title", "content"]
                                    }
                                }
                            ]
                        }
                    },
                    "script_score": {
                        "script": {
                            "source": "doc['pageRank'].value == 0 ? 1 : Math.log1p(doc['pageRank'].value * params.factor)",
                            "params": {
                                "factor": 100000
                            }
                        }
                    },
                    "boost_mode": "sum"
                }
            },
            "explain": True  # 添加这一行
        }
    )
    # 获取搜索结果
    hits = response["hits"]["hits"]
    # 格式化搜索结果
    results = [
        {
            "id": hit["_id"],
            "title": hit["_source"]["title"],
            "content": hit["_source"]["content"],
            "url": hit["_source"]["url"],
            "type": hit["_source"]["type"],
            "pageRank": hit["_source"]["pageRank"],
            "explanation": hit["_explanation"]  # 添加这一行
        }
        for hit in hits
    ]
    return results

In [22]:
results = deal_bool_query('中国恒大于2022年3月21日起停牌')

for result in results:
    print(result['title'], result['url'], result['type'])
    print(result['explanation']['value'])
    print(result['pageRank'])
    

此次，中国恒大停牌距其复牌仅一个月。此前，中国恒大于2022年3月21日起停牌，在经历了停牌超17个月后，中国恒大宣布于今年8月28日起重新复牌。 https://www.thepaper.cn/newsDetail_forward_24776842 news
44.82041
0
两年净亏损8120亿-总负债超24万亿中国恒大已资不抵债 https://www.jiemian.com/article/9760788.html news
32.080894
0
国资房企远洋集团爆雷半年巨亏近200亿票据-债券也濒临违约 http://www.stcn.com/article/detail/947042.html news
28.747662
0
中国恒大复牌首日-盘中跌90 https://wallstreetcn.com/articles/3696497 news
25.952293
0
中国恒大-恒大汽车和恒大物业在港交所集体停牌 http://03333.HK news
25.64689
0
知情人士恒驰汽车优化淘汰10-25员工停薪留职1至3个月 https://www.thepaper.cn/newsDetail_forward_20991720 news
24.574154
0
中国恒大将于周一在香港复牌 https://www.guancha.cn/GongSi/2023_08_26_706438.shtml news
24.55366
0
多股现闪崩-长春高新又因集采传闻跌停 https://weibo.cn/sinaurl?u=https%3A%2F%2Fwww.caixin.com%2F2023-06-09%2F102064128.html news
23.724459
0
多所港澳台高校发布通知知网4月1日起将暂停提供部分服务 https://www.thepaper.cn/newsDetail_forward_22523161 news
21.979973
0
h-12国台办海关总署今起恢复台湾地区冰鲜白带鱼-冻竹荚鱼输入 https://news.sina.com.cn/o/2023-03-15/doc-imykxtwx4802521.shtml news
21.541649
0


In [139]:
import math

print(math.log1p(0.00039118477103319317*100000))
print(math.log1p(3.9509503613941765e-05*100000))

print(math.log1p(0.00039118477103319317*10000000))
print(math.log1p(3.9509503613941765e-05*10000000))

3.691837003831898
1.5995795503532988
8.272020702549892
5.9816541727124815
